# Landsat NDVI and NDFI

In [1]:
import matplotlib.pyplot as plt
import rasterio

from vegmapper import landsat
from vegmapper import pathurl

## User Inputs

In [2]:
# Site name
sitename = 'ucayali'

# Project directory (local path or cloud bucket URL)
proj_dir = sitename

# AOI file
aoifile = f'{proj_dir}/ucayali_boundary.geojson'

# Reference tiles
tiles = f'{proj_dir}/{sitename}_tiles.geojson'

# AOI tiles
aoi_tiles = f'{proj_dir}/{sitename}_tiles.geojson'

# Resolution
res = 30

#Year of interest for NDFI bands
year = 2024

# Year of interest for disturbance detection
start_year = 2000 #start year of disturbance detection
end_year = 2024 # end year of disturbance detection

#Define endmembers for spectral mixture analysis.
sma_endmembers = {
            'gv':    [0.05, 0.09, 0.04, 0.61, 0.3, 0.1],
            'shade': [0.0,  0.0,  0.0,  0.0,  0.0, 0.0],
            'npv':   [0.14, 0.17, 0.22, 0.3,  0.55, 0.3],
            'soil':  [0.2,  0.3,  0.34, 0.58, 0.6,  0.58],
            'cloud': [0.9,  0.96, 0.8,  0.78, 0.72, 0.65],
        }
# Threshold of NDFI magnitude change (negative NDFI magnitude change)
threshold = -0.1

#Forest mask (If needed)
forestMask = 'Your Forest Mask Asset'

# Google Storage URL
gs = 'gs://hasan_gee_export'

# Earth Engine asses ID (Table ID) for training points
training_sample = 'projects/ee-hasanresearch/assets/TrainingPoints_Ucayali2015'

## Landsat-8 & 9 NDVI

Downloading the Landsat data, masking clouds, computing the NDVI (Normalized Difference Vegetation Index), and extracting the highest NDVI value per pixel across the year. These processing steps are handled using Google Earth Engine. The results can be exported to your Google Storage bucket, or your Google Drive if you don't have access to a Google Storage bucket.

### `landsat.‎export_landsat_ndvi`

```
landsat.export_landsat_ndvi(proj_dir, sitename, aoi_tiles, res, year, gs)
```

Paremeters:

|Paremeters|Description|Required|Default|
|----|----|----|----|
|sitename|Site name|Yes||
|aoi_tiles|AOI tiles prepared in the first notebook|Yes||
|res|Resolution|Yes||
|year|Year|Yes||
|gs|Google Storage bucket URL gs://[your gs bucket]|No|None|

In [ ]:
landsat_tasks = landsat.export_landsat_ndvi(proj_dir, sitename, aoi_tiles, res, year, gs=None)

In [ ]:
for task in landsat_tasks:
    status = task.status()
    print(f"{status['description']}: {status['state']}")

# Landsat 8 & Landsat 9 NDFI

Downloading the Landsat data, masking clouds, extracting the median, and computing the NDFI (Normalized Difference Fragment Index). These processing steps are handled using Google Earth Engine. The results can be exported to your Google Storage bucket, or your Google Drive if you don't have access to a Google Storage bucket.

Reference for NDFI and SMA: https://doi.org/10.1016/j.rse.2005.07.013

### `landsat.export_landsat_ndfi`

```
landsat.export_landsat_ndfi(proj_dir, sitename, tiles, res, year, sma_endmembers, gs=None)
```

Paremeters:

|Paremeters|Description|Required|Default|
|----|----|----|----|
|sitename|Site name|Yes||
|tiles|AOI tiles prepared in the first notebook|Yes||
|res|Resolution|Yes||
|year|Year|Yes||
|sma_endmembers|Endmembers for Spectral Mixture Analysis|For other regions||
|gs|Google Storage bucket URL gs://[your gs bucket]|No|None|


sma_endmembers:

|Endmember types|||||||
|----|----|----|----|----|----|----|
|gv|0.05| 0.09| 0.04| 0.61| 0.3| 0.1|
|shade|0.0|  0.0|  0.0|  0.0|  0.0| 0.0|
|npv|0.14| 0.17| 0.22| 0.3|  0.55|, 0.3|
|soil|0.2|  0.3|  0.34| 0.58| 0.6|  0.58|
|cloud|0.9|  0.96| 0.8|  0.78| 0.72| 0.65|

In [3]:
ndfi_tasks = landsat.export_landsat_ndfi(proj_dir, sitename, tiles, res, year, sma_endmembers, gs=None)

## Check GEE Task Status
To check the GEE task status, you can either run the following cell or directly check on Earth Engine Task Manager: https://code.earthengine.google.com/tasks. You will need to wait until all task status shown as COMPLETED before downloading them to project directory.

In [4]:
for task in ndfi_tasks:
    status = task.status()
    print(f"{status['description']}: {status['state']}")